In [1]:
import sqlite3
from sqlite3 import Error
import os.path
from os import path

#from create_db import create_connection, create_table, clean_column_names
#from populate_db import extract_data, insert_records
#import query_db as qd
import config
import build_dataset as build
import pipeline
import classification

import importlib
import datetime
import re
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_score, PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix, classification_report, plot_roc_curve, plot_precision_recall_curve 

# Decision Tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [59]:
importlib.reload(build)
importlib.reload(config)
importlib.reload(pipeline)
importlib.reload(classification)

<module 'classification' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/classification.py'>

# Notebook Table of Contents:
1. Get and fit the best model
3. Import active sentence data
2. Predict recidivism levels using active sentences and active (almost complete) sentences
4. Store results and graphs

In [82]:
# Import best models
model = 'DecisionTreeClassifier'
best_models = pd.read_csv('../results/1994/best_models_binary_Demographics.csv')

In [83]:
best_models.head()

,classifier,test_accuracy,test_precision,test_recall,test_f1
0,"LogisticRegression(C=0.01, class_weight=None, ...",0.718347,0.586271,0.255417,0.355817
1,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",0.699838,0.511788,0.312463,0.388025


In [84]:
model_to_predict = best_models[best_models['classifier'].str.contains(classifier)]['classifier'].values[0]
print("Using this model to fit and predict active sentences...\n\n", model_to_predict)              

Using this model to fit and predict active sentences...

 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')


In [85]:
len('DecisionTreeClassifier(')
params_string = model_to_predict[len('DecisionTreeClassifier('):-1]
#params_string

In [86]:
split_params = params_string.replace('=', ':')
split_params = split_params.replace('\n', '')
#split_params

In [87]:
clean_split = [param.strip() for param in split_params[1:-1].split(', ')]
#clean_split

In [88]:
dict_params = {param.split(":")[0]: param.split(":")[1] for param in clean_split} 
#dict_params

In [89]:
keys = ['criterion', 'max_depth', 'min_samples_split']
best_params = {key: dict_params[key] for key in keys}

best_params['criterion'] = best_params['criterion'][1:-1]
best_params['max_depth'] = int(best_params['max_depth'])
best_params['min_samples_split'] = int(best_params['min_samples_split'])

best_params

{'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 100}

In [90]:
# Fit the classifier
best_model = DecisionTreeClassifier(criterion=best_params['criterion'], 
                                    max_depth=best_params['max_depth'],
                                    min_samples_split=best_params['min_samples_split'],
                                    random_state=config.randomState)

In [3]:
# Import data - trimmed to 1994 here
dataset_3 = pd.read_csv('../data/dataset_main_active3.csv')

/Users/daminisharma/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [92]:
# Final selection
target = config.target_vars[0]
target_type = "binary"
features = "Demographics"

train_data, test_data, validate_data, active_sentences, active_almost_complete = build.split_and_process(dataset_3,
                                                                                                         config,
                                                                                                         target_type,
                                                                                                         features)

# Sanity Check
print('Train and Active Check:')
build.sanity_check(train_data,active_sentences)

print('Train and Active Almost Complete Check:')
build.sanity_check(train_data,active_almost_complete)

248
Size of active sentences dataset:  31360
Dataset size:  527096


//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

Total Number of Unique IDs: 304786
Total Number of IDs in Test Data: 60957
Total Number of IDs in Train Data: 195063
Total Number of IDs in Validate Data: 48766
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.19999934380188067
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8000006561981193
Does Validate Represent 20% of the Train+Validate Data?: 0.20000082024697635
Does Train Represent 80% of the Train+Validate Data?: 0.7999991797530236


/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

Index(['RACE', 'GENDER', 'ETHNICITY', 'CONTROL_STATUS', 'OFFENSE_CLEAN',
       'Current_Offense_Risk_Level', 'year_month', 'INFRACTION_PER_SENT',
       'misd_count', 'felon_count', 'sentence_count', 'age_at_sentence',
       'unemp_rate', 'ID', 'COMMITMENT_PREFIX', 'BIRTH_DATE',
       'EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')


//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Train and Active Check:
Success: Features match
Train and Active Almost Complete Check:
Success: Features match


In [76]:
# Fit and predict for 1994
print("Running: ", best_model)
best_model.fit(train_data.loc[:,train_data.columns!=target], train_data[target])

predict_active = best_model.predict(active_sentences.loc[:,active_sentences.columns!=target])
predict_almost_complete = best_model.predict(active_almost_complete.loc[:,active_almost_complete.columns!=target])

Running:  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')


In [79]:
predict_active.sum() / len (predict_active)

0.1564094387755102

In [78]:
predict_almost_complete.sum() / len(predict_almost_complete)

0.18798882681564247

In [97]:
# Run this for 2008 - make sure to change the value in trim_data()
train_data, test_data, validate_data, active_sentences, active_almost_complete = build.split_and_process(dataset_3,
                                                                                                         config,
                                                                                                         target_type,
                                                                                                         features)

# Sanity Check
print('Train and Active Check:')
build.sanity_check(train_data,active_sentences)

print('Train and Active Almost Complete Check:')
build.sanity_check(train_data,active_almost_complete)

print("Running: ", best_model)
best_model.fit(train_data.loc[:,train_data.columns!=target], train_data[target])

predict_active = best_model.predict(active_sentences.loc[:,active_sentences.columns!=target])
predict_almost_complete = best_model.predict(active_almost_complete.loc[:,active_almost_complete.columns!=target])

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


248
Size of active sentences dataset:  27486
Dataset size:  224981


//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

Total Number of Unique IDs: 164194
Total Number of IDs in Test Data: 32839
Total Number of IDs in Train Data: 105084
Total Number of IDs in Validate Data: 26271
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.2000012180713059
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.7999987819286941
Does Validate Represent 20% of the Train+Validate Data?: 0.2
Does Train Represent 80% of the Train+Validate Data?: 0.8


/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

Index(['RACE', 'GENDER', 'ETHNICITY', 'CONTROL_STATUS', 'OFFENSE_CLEAN',
       'Current_Offense_Risk_Level', 'year_month', 'INFRACTION_PER_SENT',
       'misd_count', 'felon_count', 'sentence_count', 'age_at_sentence',
       'unemp_rate', 'ID', 'COMMITMENT_PREFIX', 'BIRTH_DATE',
       'EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')


//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Train and Active Check:
Success: Features match
Train and Active Almost Complete Check:
Success: Features match
Running:  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')


In [98]:
# IGNORE
#predict_active.sum() / len (predict_active)

0.06028523612020665

In [99]:
# IGNORE
#predict_almost_complete.sum() / len(predict_almost_complete)

0.07733107032990347

In [4]:
# Check if function works for 1994
classifier = 'DecisionTreeClassifier'
results_path = '../results/1994/best_models_binary_Demographics.csv'
target = config.target_vars[0]
target_type = "binary"
features = "Demographics"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features)

248
Size of active sentences dataset:  31360
Dataset size:  527096


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

Total Number of Unique IDs: 304786
Total Number of IDs in Test Data: 60957
Total Number of IDs in Train Data: 195063
Total Number of IDs in Validate Data: 48766
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.19999934380188067
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8000006561981193
Does Validate Represent 20% of the Train+Validate Data?: 0.20000082024697635
Does Train Represent 80% of the Train+Validate Data?: 0.7999991797530236


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['RACE', 'GENDER', 'ETHNICITY', 'CONTROL_STATUS', 'OFFENSE_CLEAN',
       'Current_Offense_Risk_Level', 'year_month', 'INFRACTION_PER_SENT',
       'misd_count', 'felon_count', 'sentence_count', 'age_at_sentence',
       'unemp_rate', 'ID', 'COMMITMENT_PREFIX', 'BIRTH_DATE',
       'EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')


In [7]:
active_prob_1994, almost_complete_prob_1994 = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete, 1994)


Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Likelihood of recidivism (all active sentences) for 1994 : 0.19049744897959184
Likelihood of recidivism (almost complete active sentences) for 1994 : 0.211731843575419


In [9]:
dataset_no_active_norm.groupby(target).count()

,INFRACTION_PER_SENT,misd_count,felon_count,sentence_count,age_at_sentence,unemp_rate,age_first_offense,juv_first_offense,current_crime_violent,year_month_2005-07,...,age_cat_30-34,year_month_2008-07,year_month_1998-01,OFFENSE_CLEAN_TRAFFICKING SCHEDULE VI,year_month_2006-12,year_month_2017-12,year_month_2002-04,OFFENSE_CLEAN_DWI LEVEL 5,year_month_2018-07,year_month_1995-12
Recidivate_Risk_Level,,,,,,,,,,,,,,,,,,,,,
0.0,366535,366535,366535,366535,366535,366535,366535,366535,366535,366535,...,366535,366535,366535,366535,366535,366535,366535,366535,366535,366535
1.0,161094,161094,161094,161094,161094,161094,161094,161094,161094,161094,...,161094,161094,161094,161094,161094,161094,161094,161094,161094,161094


In [11]:
active_almost_complete.shape

(10740, 454)

In [12]:
0.211731843575419 * active_almost_complete.shape[0]

2274.0

In [55]:
#results_path = '../results/1994/best_models_binary_Demographics.csv'

In [13]:
# Check if function works for 1994
classifier = 'DecisionTreeClassifier'
results_path = '../results/1994/best_models_binary_Demographics.csv'
target = config.target_vars[0]
target_type = "all"
features = "Demographics"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features)

248
Size of active sentences dataset:  31360
Dataset size:  527096
Total Number of Unique IDs: 304786
Total Number of IDs in Test Data: 60957
Total Number of IDs in Train Data: 195063
Total Number of IDs in Validate Data: 48766
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.19999934380188067
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8000006561981193
Does Validate Represent 20% of the Train+Validate Data?: 0.20000082024697635
Does Train Represent 80% of the Train+Validate Data?: 0.7999991797530236


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['Recidivate_Risk_Level', 'RACE', 'GENDER', 'ETHNICITY',
       'CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'END_DATE', 'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'age_cat',
       'age_first_offense', 'juv_first_offense', 'current_crime_violent'],
      dtype='object')


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
dataset_no_active_norm.groupby(target).count()

,INFRACTION_PER_SENT,misd_count,felon_count,sentence_count,age_at_sentence,unemp_rate,age_first_offense,juv_first_offense,current_crime_violent,year_month_2005-07,...,age_cat_30-34,year_month_2008-07,year_month_1998-01,OFFENSE_CLEAN_TRAFFICKING SCHEDULE VI,year_month_2006-12,year_month_2017-12,year_month_2002-04,OFFENSE_CLEAN_DWI LEVEL 5,year_month_2018-07,year_month_1995-12
Recidivate_Risk_Level,,,,,,,,,,,,,,,,,,,,,
0.0,366535,366535,366535,366535,366535,366535,366535,366535,366535,366535,...,366535,366535,366535,366535,366535,366535,366535,366535,366535,366535
1.0,37290,37290,37290,37290,37290,37290,37290,37290,37290,37290,...,37290,37290,37290,37290,37290,37290,37290,37290,37290,37290
2.0,43981,43981,43981,43981,43981,43981,43981,43981,43981,43981,...,43981,43981,43981,43981,43981,43981,43981,43981,43981,43981
3.0,61866,61866,61866,61866,61866,61866,61866,61866,61866,61866,...,61866,61866,61866,61866,61866,61866,61866,61866,61866,61866
4.0,15530,15530,15530,15530,15530,15530,15530,15530,15530,15530,...,15530,15530,15530,15530,15530,15530,15530,15530,15530,15530
5.0,2427,2427,2427,2427,2427,2427,2427,2427,2427,2427,...,2427,2427,2427,2427,2427,2427,2427,2427,2427,2427


In [53]:
#((2427+15530) / dataset_no_active_norm.shape[0])

In [54]:
#((2427+15530) / dataset_no_active_norm.shape[0])*2274

In [19]:
train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features)



248
Size of active sentences dataset:  31360
Dataset size:  527096
Total Number of Unique IDs: 304786
Total Number of IDs in Test Data: 60957
Total Number of IDs in Train Data: 195063
Total Number of IDs in Validate Data: 48766
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.19999934380188067
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8000006561981193
Does Validate Represent 20% of the Train+Validate Data?: 0.20000082024697635
Does Train Represent 80% of the Train+Validate Data?: 0.7999991797530236


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['Recidivate_Risk_Level', 'RACE', 'GENDER', 'ETHNICITY',
       'CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'END_DATE', 'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'age_cat',
       'age_first_offense', 'juv_first_offense', 'current_crime_violent'],
      dtype='object')


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
dataset_no_active_norm.groupby(target).count()

,INFRACTION_PER_SENT,misd_count,felon_count,sentence_count,age_at_sentence,unemp_rate,age_first_offense,juv_first_offense,current_crime_violent,year_month_2005-07,...,age_cat_30-34,year_month_2008-07,year_month_1998-01,OFFENSE_CLEAN_TRAFFICKING SCHEDULE VI,year_month_2006-12,year_month_2017-12,year_month_2002-04,OFFENSE_CLEAN_DWI LEVEL 5,year_month_2018-07,year_month_1995-12
Recidivate_Risk_Level,,,,,,,,,,,,,,,,,,,,,
0.0,366535,366535,366535,366535,366535,366535,366535,366535,366535,366535,...,366535,366535,366535,366535,366535,366535,366535,366535,366535,366535
1.0,37290,37290,37290,37290,37290,37290,37290,37290,37290,37290,...,37290,37290,37290,37290,37290,37290,37290,37290,37290,37290
2.0,43981,43981,43981,43981,43981,43981,43981,43981,43981,43981,...,43981,43981,43981,43981,43981,43981,43981,43981,43981,43981
3.0,61866,61866,61866,61866,61866,61866,61866,61866,61866,61866,...,61866,61866,61866,61866,61866,61866,61866,61866,61866,61866
4.0,15530,15530,15530,15530,15530,15530,15530,15530,15530,15530,...,15530,15530,15530,15530,15530,15530,15530,15530,15530,15530
5.0,2427,2427,2427,2427,2427,2427,2427,2427,2427,2427,...,2427,2427,2427,2427,2427,2427,2427,2427,2427,2427


In [52]:
active_prob_1994, almost_complete_prob_1994 = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, 1994)



Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Number of active sentences for 1994 : 31360
Number of active sentences for 1994 : 10740
Likelihood of recidivism (all active sentences) for 1994 :
   group  count       pct
0    0.0  30463  0.971397
1    1.0     25  0.000797
2    2.0     92  0.002934
3    3.0    131  0.004177
4    4.0    649  0.020695


Likelihood of recidivism (almost complete active sentences) for 1994 :
   group  count       pct
0    0.0  10507  0.978305
1    1.0     14  0.001304
2    2.0     43  0.004004
3    3.0     30  0.002793
4    4.0    146  0.013594


In [ ]:
#predict_active_df = pd.DataFrame(predict_active)

In [ ]:
#predict_active_df = predict_active_df.rename(columns={0:'group'})

In [42]:
# total = predict_active_df.groupby('group').size().reset_index(name="count")
# total['pct'] = total['count'] / predict_active_df.shape[0]

In [ ]:
# total

In [ ]:
# 2% - level 4
# 0% - level 5


In [60]:
# Run train/test for 1994 binary demographics
# Run predict for 1994 binary demographics

classifier = 'DecisionTreeClassifier'
results_path = '../results/1994/best_models_binary_Demographics.csv'
target = config.target_vars[0]
features = "Demographics"
target_type = "binary"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 1994 three_class demographics
# Run predict for 1994 three_class demographics


target_type = "three_class"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 1994 six_class demographics
# Run predict for 1994 six_class demographics

target_type = "all"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 2008 binary demographics
# Run predict for 2008 binary demographics

target_type = "binary"
year = 2008

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)

# Run train/test for  2008 three_class demographics
# Run predict for 2008 three_class demographics

target_type = "three_class"
year = 2008

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 2008 six_class demographics
# Run predict for 2008 six_class demographics

target_type = "all"
year = 2008

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)





248
Size of active sentences dataset:  31360
Dataset size:  527096


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

Total Number of Unique IDs: 304786
Total Number of IDs in Test Data: 60957
Total Number of IDs in Train Data: 195063
Total Number of IDs in Validate Data: 48766
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.19999934380188067
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8000006561981193
Does Validate Represent 20% of the Train+Validate Data?: 0.20000082024697635
Does Train Represent 80% of the Train+Validate Data?: 0.7999991797530236


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['RACE', 'GENDER', 'ETHNICITY', 'CONTROL_STATUS', 'OFFENSE_CLEAN',
       'Current_Offense_Risk_Level', 'year_month', 'INFRACTION_PER_SENT',
       'misd_count', 'felon_count', 'sentence_count', 'age_at_sentence',
       'unemp_rate', 'ID', 'COMMITMENT_PREFIX', 'BIRTH_DATE',
       'EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')
Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Number of active sen

...

Index(['RACE', 'GENDER', 'ETHNICITY', 'CONTROL_STATUS', 'OFFENSE_CLEAN',
       'Current_Offense_Risk_Level', 'year_month', 'INFRACTION_PER_SENT',
       'misd_count', 'felon_count', 'sentence_count', 'age_at_sentence',
       'unemp_rate', 'ID', 'COMMITMENT_PREFIX', 'BIRTH_DATE',
       'EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')
Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Number of active sen

/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['Recidivate_Risk_Level', 'RACE', 'GENDER', 'ETHNICITY',
       'CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'END_DATE', 'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'age_cat',
       'age_first_offense', 'juv_first_offense', 'current_crime_violent'],
      dtype='object')


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Number of active sentences for 2008 : 27486
Number of active sentences for 2008 : 9427
Likelihood of recidivism (all active sentences) for 2008 :
   group  count  pct
0    0.0  27486  1.0


Likelihood of recidivism (almost complete active sentences) for 2008 :
   group  count  pct
0    0.0   9427  1.0
